## Setup

In [19]:
import pandas as pd
import numpy as np
import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate


In [2]:
# packages used by helper functions
import uuid

In [3]:
# packages for prompting definitions
import sys
sys.path.append("..")
import json

In [6]:
from groq import Groq
groq_api_key="api key"

chat = ChatGroq(temperature=0, groq_api_key="api key", model_name="mixtral-8x7b-32768")


## Prompt definitions (function to extract triplets in JSON-format for knowledge graph from text chunks)

In [51]:
#################################
# Definition of used LLM
#################################
##########################################################################
def graphPrompt(input: str, metadata={}, model="mistral:latest"):
    if model == None:
        model = "mistral:latest"
    
    chunk_id = metadata.get('chunk_id', None)

    # model_info = client.show(model_name=model)
    # print( chalk.blue(model_info))

    SYS_PROMPT = ("You are a network graph maker who extracts terms and their relations from a given context. "
        "You are provided with a context chunk (delimited by ```) Your task is to extract the ontology "
        "of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n"
        "Thought 1: While traversing through each sentence, Think about the key terms mentioned in it.\n"
            "\tTerms may include requirement (AFO), rfc, organization, date, duration, \n"
            "\tcondition, concept, object, entity  etc.\n"
            "\tTerms should be as atomistic as possible\n\n"
        "Thought 2: Think about how these terms can have one on one relation with other terms.\n"
            "\tTerms that are mentioned in the same sentence or the same paragraph are typically related to each other.\n"
            "\tTerms can be related to many other terms\n\n"
        "Thought 3: Find out the relation between each such related pair of terms. \n\n"
        "Format your output as a list of json. Each element of the list contains a pair of terms"
        "and the relation between them like the follwing. NEVER change the value of the chunk_ID as defined in this prompt: \n"
        "[\n"
        "   {\n"
        '       "chunk_id": "CHUNK_ID_GOES_HERE",\n'
        '       "node_1": "A concept from extracted ontology",\n'
        '       "node_2": "A related concept from extracted ontology",\n'
        '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n' 
        "   }, {...}\n"
        "]"
    )
    SYS_PROMPT = SYS_PROMPT.replace('CHUNK_ID_GOES_HERE', chunk_id)

    USER_PROMPT = f"context: ```{input}``` \n\n output: "
    print("USER_PROMPT promt\n")
    print(USER_PROMPT)
    #response = client.generate(model="mistral:latest", system=SYS_PROMPT, prompt=USER_PROMPT)
    human = "context: ```{input}``` \n\n output: "
    prompt = ChatPromptTemplate.from_messages([("system", SYS_PROMPT), ("human", human)])
    print("Prompt\n")
    print(prompt)
    chain = prompt | chat
    print("cbain\n")
    print(chain)
    aux1=chain.invoke({"input":input})
    print("aux\n")
    print(aux1)

    #aux1 = response['response']
    # Find the index of the first open bracket '['
    start_index = aux1.find('[')
    # Slice the string from start_index to extract the JSON part and fix an unexpected problem with insertes escapes (WHY ?)
    json_string = aux1[start_index:]
    json_string = json_string.replace('\\\\\_', '_')
    json_string = json_string.replace('\\\\_', '_')
    json_string = json_string.replace('\\\_', '_')
    json_string = json_string.replace('\\_', '_')
    json_string = json_string.replace('\_', '_')
    json_string.lstrip() # eliminate eventual leading blank spaces
#####################################################
    print("json-string:\n" + json_string)
#####################################################         
    try:
        result = json.loads(json_string)
        result = [dict(item) for item in result]
    except:
        print("\n\nERROR ### Here is the buggy response: ", response, "\n\n")
        result = None
    print("§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§")

    return result

## Functions

In [8]:
def documents2Dataframe(documents) -> pd.DataFrame:
    rows = []
    for chunk in documents:
        row = {
            "text": chunk.page_content,
            **chunk.metadata,
            "chunk_id": uuid.uuid4().hex,
        }
        rows = rows + [row]

    df = pd.DataFrame(rows)
    return df

In [9]:
def df2Graph(dataframe: pd.DataFrame, model=None) -> list:
    # dataframe.reset_index(inplace=True)
    results = dataframe.apply(
        lambda row: graphPrompt(row.text, {"chunk_id": row.chunk_id}, model), axis=1
    )
    # invalid json results in NaN
    results = results.dropna()
    results = results.reset_index(drop=True)

    ## Flatten the list of lists to one single list of entities.
    concept_list = np.concatenate(results).ravel().tolist()
    return concept_list

In [10]:
def graph2Df(nodes_list) -> pd.DataFrame:
    ## Remove all NaN entities
    graph_dataframe = pd.DataFrame(nodes_list).replace(" ", np.nan)
    graph_dataframe = graph_dataframe.dropna(subset=["node_1", "node_2"])
    graph_dataframe["node_1"] = graph_dataframe["node_1"].apply(lambda x: x.lower())
    graph_dataframe["node_2"] = graph_dataframe["node_2"].apply(lambda x: x.lower())

    return graph_dataframe

In [11]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2

## Variables

In [12]:
import pdfplumber
## Input data directory
##########################################################
input_file_name = "gemSpec_VZD_V1.14.0.pdf"
##########################################################
data_dir = "./HotG_Data/"+input_file_name
inputdirectory = Path(f"./{data_dir+'.txt'}")

with pdfplumber.open(data_dir) as pdf:
    # Leeren String zum Speichern des Texts
    text = ''
    
    # Über alle Seiten iterieren
    for page in pdf.pages:
        # Text der aktuellen Seite extrahieren und an den Gesamttext anhängen
        text += page.extract_text()

# Text in eine TXT-Datei schreiben
with open(data_dir+'.txt', 'w', encoding='utf-8') as txt_file:
    txt_file.write(text)

## This is where the output csv files will be written
outputdirectory = Path(f"./data_output")

print(inputdirectory)
print(input_file_name)

HotG_Data/gemSpec_VZD_V1.14.0.pdf.txt
gemSpec_VZD_V1.14.0.pdf


In [13]:
input_file_name=input_file_name+'.txt'
output_graph_file_name = f"graph_{input_file_name[:-4]}.csv"
output_graph_file_with_path = outputdirectory/output_graph_file_name

output_chunks_file_name = f"chunks_{input_file_name[:-4]}.csv"
output_chunks_file_with_path = outputdirectory/output_chunks_file_name

output_context_prox_file_name = f"graph_contex_prox_{input_file_name[:-4]}.csv"
output_context_prox_file_with_path = outputdirectory/output_context_prox_file_name

print(output_graph_file_with_path)
print(output_chunks_file_with_path)
print(output_context_prox_file_with_path)

data_output/graph_gemSpec_VZD_V1.14.0.pdf.csv
data_output/chunks_gemSpec_VZD_V1.14.0.pdf.csv
data_output/graph_contex_prox_gemSpec_VZD_V1.14.0.pdf.csv


## Load Documents

In [14]:
#loader = TextLoader("./HotG_Data/Hanse.txt")
loader = TextLoader(inputdirectory)
Document = loader.load()
# clean unnecessary line breaks
Document[0].page_content = Document[0].page_content.replace("\n", " ")

In [15]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(Document)
print("Number of chunks = ", len(pages))
print(pages[5].page_content)

Number of chunks =  153
Umsetzung ......................................................................................16 4.2.1.2 Nutzung ..........................................................................................19 4.2.2 Operation read_Directory_Entry ...............................................................20 4.2.2.1 Umsetzung ......................................................................................20 4.2.2.2 Nutzung ..........................................................................................21 4.2.3 Operation modify_Directory_Entry ...........................................................22 4.2.3.1 Umsetzung ......................................................................................22 4.2.3.2 Nutzung ..........................................................................................22 4.2.4 Operation delete_Directory_Entry ............................................................23 4.2.4.1 Umsetzung


## Create a dataframe of all the chunks

In [16]:
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(153, 3)


,text,source,chunk_id
0,Elektronische Gesundheitskarte und Telematikin...,HotG_Data/gemSpec_VZD_V1.14.0.pdf.txt,5aac2d439dcc42a88aad0f6967f26b4a
1,Einarbeitung lt. Änderungsliste gematik 1.5.0 ...,HotG_Data/gemSpec_VZD_V1.14.0.pdf.txt,8fa81b4752d3402481adc21ebffc6a07
2,KIM 1.5.1 gemSpec_VZD_V1.docx Spezifikation Se...,HotG_Data/gemSpec_VZD_V1.14.0.pdf.txt,151c9bc0b01142a49d7590a88bc2bb48
3,Einordnung des Dokumentes .......................,HotG_Data/gemSpec_VZD_V1.14.0.pdf.txt,b0691d9054ad450bb3b94edbb46ffbf1
4,.................................................,HotG_Data/gemSpec_VZD_V1.14.0.pdf.txt,bfe87fdd6a2245c1abb887228f5e62e5


## Extract Concepts

In [52]:
## To regenerate the graph with LLM, set this to True
##################
regenerate = True  # toggle to True if the time-consuming (re-)generation of the knowlege extraction is required
##################
if regenerate:
#########################################################    
    concepts_list = df2Graph(df, model='mistral:latest')
#########################################################
    dfg1 = graph2Df(concepts_list)
    
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(output_graph_file_with_path, sep=";", index=False)
    df.to_csv(output_chunks_file_with_path, sep=";", index=False)
else:
    dfg1 = pd.read_csv(output_graph_file_with_path, sep=";")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
dfg1.head()

USER_PROMPT promt

context: ```Elektronische Gesundheitskarte und Telematikinfrastruktur Spezifikation Verzeichnisdienst Version: 1.14.0 Revision: 434138 Stand: 31.01.2022 Status: freigegeben Klassifizierung: öffentlich Referenzierung: gemSpec_VZD gemSpec_VZD_V1.docx Spezifikation Seite 1 von 72 Version: 1.14.0 © gematik – öffentlich Stand: 31.01.2022Spezifikation Verzeichnisdienst Dokumentinformationen Änderungen zur Vorversion Anpassungen des vorliegenden Dokumentes im Vergleich zur Vorversion können Sie der nachfolgenden Tabelle entnehmen. Bitte beachten Sie die Hinweise zur Einführung der Benennungen 'WANDA Basic' und 'WANDA Smart' (siehe Dokumentenhistorie). Dokumentenhistorie Version Stand Kap./ Grund der Änderung, besondere Hinweise Bearbeitung Seite 1.2.0 17.07.15 Nutzer der Schnittstelle gematik I_Directory_Maintenance geändert 1.3.0 24.08.16 Anpassungen zum Online-Produktivbetrieb gematik (Stufe 1) 1.4.0 28.10.16 Einarbeitung lt. Änderungsliste gematik 1.5.0 19.04.17 Anpassun

KeyError: 'Input to ChatPromptTemplate is missing variables {\'...\', \'\\n       "chunk_id"\'}.  Expected: [\'\\n       "chunk_id"\', \'...\', \'input\'] Received: [\'input\']'

## Calculating contextual proximity

In [ ]:
dfg2 = contextual_proximity(dfg1)
dfg2.to_csv(output_context_prox_file_with_path, sep=";", index=False)
dfg2.tail()

NameError: name 'contextual_proximity' is not defined

### Merge both the dataframes

In [ ]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,960,"dobrawa, a daughter of duke boleslaus i of boh...","8ad949a3488a430cae0864089c8cef99,8ad949a3488a4...",contextual proximity,2
1,960,margrave gero,"8ad949a3488a430cae0864089c8cef99,8ad949a3488a4...",contextual proximity,3
2,960,mieszko,"8ad949a3488a430cae0864089c8cef99,8ad949a3488a4...",contextual proximity,8
3,960,otto the great,"8ad949a3488a430cae0864089c8cef99,8ad949a3488a4...",contextual proximity,2
4,960,poland,"8ad949a3488a430cae0864089c8cef99,8ad949a3488a4...",contextual proximity,2
...,...,...,...,...,...
2729,zone of influence,early polish rulers,"9e984b449e2c49338bb8f48c0ac557f3,9e984b449e2c4...",contextual proximity,2
2730,zone of influence,neighbors,"9e984b449e2c49338bb8f48c0ac557f3,9e984b449e2c4...",contextual proximity,2
2731,zone of influence,poland,"9e984b449e2c49338bb8f48c0ac557f3,9e984b449e2c4...",contextual proximity,2
2732,zone of influence,traces of smaller fortifications,"9e984b449e2c49338bb8f48c0ac557f3,9e984b449e2c4...",contextual proximity,2


## Calculate the NetworkX Graph

In [ ]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(401,)

In [ ]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

### Calculate communities for coloring the nodes

In [ ]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  6
[['960', '970', 'apostles to the slavs', 'archaeologists', 'balkans', 'base in central bohemia', 'basilica de san clemente', 'basilica of san clemente', 'bavarian church', 'bavarian missionaries', 'being thrown overboard in the black sea with an anchor attached to his feet', 'bohemian warlords', 'boleslaus son, duke boleslaus ii', 'bořivoj', 'byzantian empire, slavonic literature and material heritage of the west-slavonic peoples', 'carolingian empire', 'carolingians', 'centers of power', 'charlemagne', 'christianization period, central europe', 'church liturgy', 'church slavonic', 'consolidated protofeudal state, ruling dynasty, castles, church, economy', 'constantinople', 'convert miesco to christianity', 'converting to christianity', 'crimea beach', 'cyrill', 'desire', 'developed later than bohemia', 'dobrawa, a daughter of duke boleslaus i of bohemia', 'early polish rulers', 'east francians', 'economic model', 'emperor in constantinople', 'erected between

### Create a dataframe for community colors

In [ ]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,960,#5f57db,1
1,970,#5f57db,1
2,apostles to the slavs,#5f57db,1
3,archaeologists,#5f57db,1
4,balkans,#5f57db,1
...,...,...,...
396,unharmed hand of poppo,#db57d3,4
397,normandy or parts of normandy,#57d3db,5
398,rollo,#57d3db,5
399,political and economic factors,#57db5f,6


### Add colors to the graph

In [ ]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [ ]:
from pyvis.network import Network

#graph_output_directory = "./docs/index.html"

net = Network(
    notebook=True,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="800px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)

# net.show(graph_output_directory)
net.show_buttons(filter_=['physics'])
net.show("knowledge_graph.html")

knowledge_graph.html


In [ ]:
# DETAILED STEPS OF TERM PROXIMITY CALCULATION (same as function, only step by step to better understand the process)

In [ ]:
## Melt the dataframe into a list of nodes
dfg_long = pd.melt(
    dfg1, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
)

In [ ]:
dfg_long.tail(5)

,chunk_id,variable,node
615,9d455bc6ec4946bf81b106a8d103c837,node_2,christian powers
616,9d455bc6ec4946bf81b106a8d103c837,node_2,superior strength
617,9d455bc6ec4946bf81b106a8d103c837,node_2,surrender or be traded south as slaves
618,9d455bc6ec4946bf81b106a8d103c837,node_2,hatred
619,9d455bc6ec4946bf81b106a8d103c837,node_2,freedom


In [ ]:
dfg_long.drop(columns=["variable"], inplace=True)
# Self join with chunk id as the key will create a link between terms occuring in the same text chunk.

In [ ]:
dfg_long.tail(5)

,chunk_id,node
615,9d455bc6ec4946bf81b106a8d103c837,christian powers
616,9d455bc6ec4946bf81b106a8d103c837,superior strength
617,9d455bc6ec4946bf81b106a8d103c837,surrender or be traded south as slaves
618,9d455bc6ec4946bf81b106a8d103c837,hatred
619,9d455bc6ec4946bf81b106a8d103c837,freedom


In [ ]:
dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))

In [ ]:
dfg_wide.head()

,chunk_id,node_1,node_2
0,3b1d78faf9064a97b1b667d4aa940745,episode 96,episode 96
1,3b1d78faf9064a97b1b667d4aa940745,episode 96,saxons
2,3b1d78faf9064a97b1b667d4aa940745,episode 96,otto i
3,3b1d78faf9064a97b1b667d4aa940745,episode 96,duchy
4,3b1d78faf9064a97b1b667d4aa940745,episode 96,original territory


In [ ]:
# drop self loops
self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
dfgraph2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)

In [ ]:
dfgraph2.head()

,chunk_id,node_1,node_2
0,3b1d78faf9064a97b1b667d4aa940745,episode 96,saxons
1,3b1d78faf9064a97b1b667d4aa940745,episode 96,otto i
2,3b1d78faf9064a97b1b667d4aa940745,episode 96,duchy
3,3b1d78faf9064a97b1b667d4aa940745,episode 96,original territory
4,3b1d78faf9064a97b1b667d4aa940745,episode 96,cities


In [ ]:
## Group and count edges.
dfgraph2 = (
    dfgraph2.groupby(["node_1", "node_2"])
    .agg({"chunk_id": [",".join, "count"]})
    .reset_index()
)

In [ ]:
dfgraph2.head()

node_1                                             node_2  \
                                                              
0    960                     convert miesco to christianity   
1    960  dobrawa, a daughter of duke boleslaus i of boh...   
2    960                                 first polish ruler   
3    960                        gniesno in the 10th century   
4    960                                      margrave gero   

                                            chunk_id        
                                                join count  
0                   8ad949a3488a430cae0864089c8cef99     1  
1  8ad949a3488a430cae0864089c8cef99,8ad949a3488a4...     2  
2                   8ad949a3488a430cae0864089c8cef99     1  
3                   8ad949a3488a430cae0864089c8cef99     1  
4  8ad949a3488a430cae0864089c8cef99,8ad949a3488a4...     3

In [ ]:
dfgraph2.columns = ["node_1", "node_2", "chunk_id", "count"]
dfgraph2.replace("", np.nan, inplace=True)
dfgraph2.dropna(subset=["node_1", "node_2"], inplace=True)
# Drop edges with 1 count
dfgraph2 = dfg2[dfg2["count"] != 1]
dfgraph2["edge"] = "contextual proximity"

In [ ]:
dfgraph2.head()

,node_1,node_2,chunk_id,count,edge
1,960,"dobrawa, a daughter of duke boleslaus i of boh...","8ad949a3488a430cae0864089c8cef99,8ad949a3488a4...",2,contextual proximity
4,960,margrave gero,"8ad949a3488a430cae0864089c8cef99,8ad949a3488a4...",3,contextual proximity
5,960,mieszko,"8ad949a3488a430cae0864089c8cef99,8ad949a3488a4...",8,contextual proximity
6,960,otto the great,"8ad949a3488a430cae0864089c8cef99,8ad949a3488a4...",2,contextual proximity
8,960,poland,"8ad949a3488a430cae0864089c8cef99,8ad949a3488a4...",2,contextual proximity


In [ ]:
dfgraph2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2640 entries, 1 to 7203
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   node_1    2640 non-null   object
 1   node_2    2640 non-null   object
 2   chunk_id  2640 non-null   object
 3   count     2640 non-null   int64 
 4   edge      2640 non-null   object
dtypes: int64(1), object(4)
memory usage: 123.8+ KB
